<a href="https://colab.research.google.com/github/xuanyu410/114-1PL-Repo/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E4%BD%9C%E6%A5%AD%E4%BA%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
# -*- coding: utf-8 -*-
# 確保這些套件已安裝: !pip install google-search-results folium geopy -q
!pip install google-search-results
import folium
from geopy.geocoders import Nominatim
from serpapi import GoogleSearch
from IPython.display import HTML # 為了 Gradio 輸出地圖
import json
import re


#請在 Colab Secrets 中設置一個名為 'serpapi_key' 的金鑰，值為 '201c232b1ea6d3e803272c07771e289ac1d651b6eb705349f1b8ba9f10f4e52d'
try:
    from google.colab import userdata
    SERPAPI_KEY = userdata.get('serpapi_key')
    if not SERPAPI_KEY:
        print("🚨 警告：SerpAPI 金鑰（serpapi_key）未從 Colab Secrets 載入，請確認設置。")
except:
    # 這裡可以手動寫入金鑰，僅用於測試，不建議在共享環境中使用
    # 請將 'YOUR_SERPAPI_KEY_HERE' 替換為您的實際金鑰
    # 提醒：請優先使用 Colab Secrets 獲取金鑰
    SERPAPI_KEY = None
    print("🚨 警告：SerpAPI 金鑰未定義。搜尋功能將失敗。")

In [75]:
# 1. 初始化地理編碼器
geolocator = Nominatim(user_agent="search_map_app")

# 2. 台灣地點列表 (省略，沿用您提供的列表)
TAIWAN_LOCATIONS = [
    # 直轄市
    '台北', '台北市', '臺北', '臺北市', 'Taipei', '新北', '新北市', 'New Taipei',
    '桃園', '桃園市', 'Taoyuan', '台中', '台中市', '臺中', '臺中市', 'Taichung',
    '台南', '台南市', '臺南', '臺南市', 'Tainan', '高雄', '高雄市', 'Kaohsiung',

    # 縣市
    '基隆', '基隆市', 'Keelung', '新竹', '新竹市', '新竹縣', 'Hsinchu',
    '苗栗', '苗栗縣', 'Miaoli', '彰化', '彰化縣', 'Changhua', '南投', '南投縣', 'Nantou',
    '雲林', '雲林縣', 'Yunlin', '嘉義', '嘉義市', '嘉義縣', 'Chiayi', '屏東', '屏東縣',
    'Pingtung', '宜蘭', '宜蘭縣', 'Yilan', '花蓮', '花蓮縣', 'Hualien', '台東', '台東縣',
    '臺東', '臺東縣', 'Taitung', '澎湖', '澎湖縣', 'Penghu', '金門', '金門縣', 'Kinmen',
    '連江', '連江縣', '馬祖', 'Matsu',

    # 知名景點/地標
    '101', '台北101', '西門町', '信義區', '中正區', '士林', '北投', '淡水',
    '九份', '平溪', '日月潭', '阿里山', '墾丁', '太魯閣', '東大門', '逢甲', '一中街', '勤美',
]

In [76]:
# 3. 搜尋相關函式 (沿用您提供的程式碼，但將 print 改為返回字串，以便 Gradio 顯示)

def search_google(query, api_key, num=5, gl='tw', hl='zh-tw'):
    try:
        params = {
            'q': query, 'api_key': api_key, 'num': num, 'gl': gl, 'hl': hl
        }
        search = GoogleSearch(params)
        results = search.get_dict()
        return results.get('organic_results', [])
    except Exception as e:
        # print(f"搜尋時發生錯誤: {e}") # 移除 print
        return []

def extract_locations_from_text(text):

    found_locations = []
    text_lower = text.lower()
    for location in TAIWAN_LOCATIONS:
        if location.lower() in text_lower:
            base_name = location.replace('市', '').replace('縣', '')
            if base_name not in [loc.replace('市', '').replace('縣', '') for loc in found_locations]:
                found_locations.append(location)
    return found_locations

def get_coordinates(location_name):

    try:
        location = geolocator.geocode(f"{location_name}, Taiwan", timeout=10)
        if location:
            return (location.latitude, location.longitude)
        return None
    except Exception as e:
        # print(f"無法取得 {location_name} 的座標: {e}") # 移除 print
        return None

def analyze_results_with_locations(results):

    analyzed_results = []
    for result in results:
        title = result.get('title', '')
        snippet = result.get('snippet', '')
        combined_text = f"{title} {snippet}"
        locations = extract_locations_from_text(combined_text)
        if locations:
            analyzed_results.append({
                'title': title,
                'snippet': snippet,
                'link': result.get('link', ''),
                'locations': locations
            })
    return analyzed_results

def create_map_from_results(analyzed_results, center=[23.5, 121.0], zoom=7):

    m = folium.Map(location=center, zoom_start=zoom, tiles='OpenStreetMap')
    marked_locations = {}
    for result in analyzed_results:
        for location in result['locations']:
            coords = get_coordinates(location)
            if coords:
                if location not in marked_locations:
                    marked_locations[location] = []
                marked_locations[location].append(result)

    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue']

    for i, (location, results) in enumerate(marked_locations.items()):
        coords = get_coordinates(location)
        if coords:
            lat, lon = coords
            popup_html = f"<div style='width: 300px;'>"
            popup_html += f"<h4 style='color: #2c3e50;'>{location}</h4>"
            popup_html += f"<p style='color: #7f8c8d; font-size: 12px;'>找到 {len(results)} 筆相關結果</p>"
            for j, res in enumerate(results, 1):
                popup_html += f"<div style='margin: 10px 0; padding: 10px; background: #ecf0f1; border-radius: 5px;'>"
                popup_html += f"<b style='color: #2980b9;'>[{j}] {res['title'][:50]}...</b><br>"
                popup_html += f"<small style='color: #34495e;'>{res['snippet'][:100]}...</small><br>"
                popup_html += f"<a href='{res['link']}' target='_blank' style='color: #3498db;'>查看連結</a>"
                popup_html += "</div>"
            popup_html += "</div>"

            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(popup_html, max_width=350),
                tooltip=f"{location} ({len(results)} 筆結果)",
                icon=folium.Icon(color=colors[i % len(colors)], icon='info-sign')
            ).add_to(m)

    return m

def display_results_summary(analyzed_results):
    """
    將分析結果摘要格式化為 Gradio 文字輸出
    """
    summary = ["="*80]
    summary.append(f"找到 {len(analyzed_results)} 筆包含地理位置的搜尋結果")
    summary.append("="*80 + "\n")

    for i, result in enumerate(analyzed_results, 1):
        summary.append(f"【結果 {i}】")
        summary.append(f"標題: {result['title']}")
        summary.append(f"地點: {', '.join(result['locations'])}")
        summary.append(f"描述: {result['snippet'][:100]}...")
        summary.append(f"網址: {result['link']}")
        summary.append("-"*80 + "\n")

    return '\n'.join(summary)

In [77]:
# 4. Gradio 介面專用的主控函式
def serpapi_search_and_map_wrapper(query, num_results=10):
    """
    整合 SerpAPI 搜尋與地圖標註，用於 Gradio 介面。
    返回地圖的 HTML 字串和結果摘要。
    """
    if not SERPAPI_KEY:
        error_msg = "錯誤：SerpAPI 金鑰（SERPAPI_KEY）未定義。請檢查您的 Colab Secrets 設置。"
        return None, error_msg

    try:
        # 1. 執行搜尋
        results = search_google(query, SERPAPI_KEY, num=num_results)

        if not results:
            return None, "沒有找到任何搜尋結果。"

        # 2. 分析結果並擷取地理位置
        analyzed_results = analyze_results_with_locations(results)

        if not analyzed_results:
            return None, "\n⚠️ 搜尋結果中沒有找到可辨識的台灣地區"

        # 3. 顯示結果摘要
        summary = display_results_summary(analyzed_results)

        # 4. 建立地圖
        map_obj = create_map_from_results(analyzed_results)

        # Folium 地圖轉為 HTML 字串
        map_html = map_obj._repr_html_()

        return map_html, summary

    except Exception as e:
        return None, f"發生錯誤: {e}"

In [78]:
# 確保您的 Gradio 介面邏輯 (wrapper_recommend_with_json, handle_row_selection, df_map_data)
# 在此區塊之前已經定義。

# 設定 Gradio 介面元件
with gr.Blocks(title="師大午餐決策器") as demo:
    gr.Markdown(
        """
        # 🍽️ 師大午餐/聚會決策器
        ### 目標：從餐廳清單中篩選，並隨機推薦 3 家高cp質餐廳，並提供台灣各地美食資訊搜尋！
        """
    )

    # 1. 餐廳決策 Tab (保留原有功能)
    with gr.Tab("🍽️ 餐廳決策與 AI 總結"):

        # 狀態元件：用於傳輸 DataFrame 資料給地圖選擇器
        recommended_df_json = gr.State(value="")

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("## 條件篩選")
                price_select = gr.CheckboxGroup(
                    ['$', '$$', '$$$'],
                    label="價位過濾 (可複選)",
                    value=['$', '$$'],
                    info="($: 銅板/150內, $$: 150-300, $$$: 300+)"
                )
                distance_select = gr.CheckboxGroup(
                    ['短', '中', '長'],
                    label="距離過濾 (可複選)",
                    value=['短', '中'],
                    info="(短: 步行5分, 中: 步行10分, 長: 需騎車)"
                )
                available_toggle = gr.Checkbox(
                    label="只看今日營業",
                    value=True
                )
                recommend_btn = gr.Button("🚀 開始推薦")

            with gr.Column(scale=2):
                gr.Markdown("## 推薦清單 (隨機 3 家)")
                output_dataframe = gr.DataFrame(
                    headers=["餐廳名稱", "價位", "距離"],
                    datatype=["str", "str", "str"],
                    label="隨機推薦結果",
                    type="pandas",
                    interactive=True,
                )
                output_list = gr.Textbox(label="推薦清單細節")

        gr.Markdown("## 📍 餐廳位置地圖 (點擊上方列表中的餐廳)")
        map_output = gr.HTML(
            # 確保 df_map_data 變數已在程式碼中定義
            generate_folium_map(None, df_map_data),
            label="互動式地圖",
            elem_id="map_container"
        )

        gr.Markdown("## 🤖 AI 決策建議 (約 100 字)")
        output_summary = gr.Textbox(
            label="AI 決策顧問報告",
            interactive=False,
            info="AI 將根據推薦結果，總結優缺點，幫助您快速做出選擇。"
        )

        # 綁定事件
        recommend_btn.click(
            fn=wrapper_recommend_with_json,
            inputs=[price_select, distance_select, available_toggle],
            outputs=[output_dataframe, output_list, output_summary, recommended_df_json]
        )

        output_dataframe.select(
            fn=handle_row_selection,
            inputs=[recommended_df_json, gr.State(df_map_data)],
            outputs=[map_output]
        )


    # 2. 台灣地圖搜尋 Tab (新增 SerpAPI 驅動功能)
    with gr.Tab("🗺️ 台灣地圖搜尋"):
        gr.Markdown("## ✨ Google 搜尋結果地圖化 (SerpAPI 驅動)")
        gr.Markdown("輸入關鍵字 (例如：台北必吃拉麵)，程式將自動解析結果中的台灣地點並在地圖上標註。")

        with gr.Row():
            query_input = gr.Textbox(label="輸入搜尋關鍵字", value="台南熱門小吃")
            num_input = gr.Slider(minimum=5, maximum=20, step=1, label="搜尋結果數量", value=10)
            search_map_btn = gr.Button("🔍 搜尋並繪製地圖")

        map_search_output = gr.HTML(
            label="搜尋結果地圖",
            elem_id="search_map_container"
        )

        summary_search_output = gr.Textbox(
            label="搜尋結果摘要",
            interactive=False,
            lines=10
        )

        # 綁定 SerpAPI 搜尋功能
        search_map_btn.click(
            fn=serpapi_search_and_map_wrapper,
            inputs=[query_input, num_input],
            outputs=[map_search_output, summary_search_output]
        )

# 運行 Gradio 應用
if __name__ == "__main__":
    print("Gradio 應用正在啟動...")
    demo.launch(inbrowser=True, share=True)

Gradio 應用正在啟動...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bdada7a680d165bb5f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
